In [1]:
## Jun code ##

import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from scipy import stats

In [3]:
with open("/Users/papiya/Desktop/CU_Capstone/Dataset/newmatrix.csv") as file:
    read_file = csv.reader(file)
    Data = list(line for line in read_file)
file.close()

In [4]:
Sample_Names = Data[0]
Site_Names = list(line[0] for line in Data[1:])
Beta_Matrix = list(list(map(float, line[1:])) for line in Data[1:])
beta_value= np.stack(Beta_Matrix)

In [5]:
m_value = np.log2(beta_value/(1 - beta_value))

with open('/Users/papiya/Desktop/CU_Capstone/Dataset/bmiStatus.csv') as file:
    read_file = csv.reader(file)
    bmi_l = list(line for line in read_file)
file.close()

with open('/Users/papiya/Desktop/CU_Capstone/Dataset/age.csv') as file:
    read_file = csv.reader(file)
    age_l = list(line for line in read_file)
file.close()

with open('/Users/papiya/Desktop/CU_Capstone/Dataset/smoking_status.csv') as file:
    read_file = csv.reader(file)
    ss_l = list(line for line in read_file)
file.close()

bmi = np.array(list(item[1] for item in bmi_l)[1:]).astype(float)
age = np.array(list(item[1] for item in age_l)[1:]).astype(float)
ss = np.array(list(item[1] for item in ss_l)[1:]).astype(float)

In [30]:
## PS changes

import pandas as pd

df_age = pd.read_csv('/Users/papiya/Desktop/CU_Capstone/Dataset/age.csv')
df_age = df_age.drop(df_age.columns[0], axis=1)
df_bmi = pd.read_csv('/Users/papiya/Desktop/CU_Capstone/Dataset/bmiStatus.csv')
df_bmi = df_bmi.drop(df_bmi.columns[0], axis=1)
df_smok = pd.read_csv('/Users/papiya/Desktop/CU_Capstone/Dataset/smoking_status.csv')
df_smok = df_smok.drop(df_smok.columns[0], axis=1)
df_smok = df_smok[df_smok.columns[0]].astype('object')
s = pd.Series(list('012')) # one hot encoding 'smoking status'
df_smok = pd.get_dummies(df_smok)

#pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          #keys=None, levels=None, names=None, verify_integrity=False,
          #copy=True)
df = pd.concat([df_age, df_bmi, df_smok], axis=1, ignore_index=True)
df.columns = ['Age', 'BMI', 'Smoke_0', 'Smoke_1', 'Smoke_2']
df.head(2)

#enc = OneHotEncoder()
#enc.fit(ss)

,Age,BMI,Smoke_0,Smoke_1,Smoke_2
0,42.69,18.7,0,0,1
1,67.75,35.9,1,0,0


#To construct a independent variable matrix with 1, age, bmi, smoking_status 1 and 2 as the five features 
#(smoking status 0 dropped so matrix invertible)

enc = OneHotEncoder()
enc.fit(np.expand_dims(np.unique(ss),1))
new_ss = enc.transform(np.expand_dims(ss, 1)).toarray()
X = np.hstack([np.vstack([np.ones(bmi.shape[0]), age, bmi]).T, new_ss[:,1:]])

In [31]:
X = df.as_matrix()
X[:2]

array([[ 42.69,  18.7 ,   0.  ,   0.  ,   1.  ],
       [ 67.75,  35.9 ,   1.  ,   0.  ,   0.  ]])

In [32]:
sample_id = Sample_Names
site_id = Site_Names
beta_values = beta_value
m_values = m_value
X = X

beta_range = np.linspace(0,1,200)
for i in range(beta_values.shape[1]):
    density = gaussian_kde(beta_values[:,i])
    #density.covariance_factor = lambda : .25
    #density._compute_covariance()
    plt.plot(beta_range, density(beta_range),'g')
plt.show()

In [33]:
lr_m = LinearRegression(fit_intercept= False, n_jobs= -1)
lr_m.fit(X, m_values.T)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1, normalize=False)

In [34]:
mse = np.sum((lr_m.predict(X) - m_values.T)**2 , axis = 0)/ float(X.shape[0] - X.shape[1])
"""
standard error is the square root of the variance of the estimate, which is the element in the diagnol of
the covariance matrix of the estimate. Use MSE as the estimated value of variance of the error. 
"""
se = np.array([np.sqrt(np.diagonal(mse[i] * np.linalg.inv(np.dot(X.T, X)) )) for i in range(mse.shape[0])]) 
t_m = lr_m.coef_/ se
p_m = 2 * (1 - stats.t.cdf(np.abs(t_m), X.shape[0] - X.shape[1]))

msr = np.sum((lr_m.predict(X) - np.mean(m_values.T, axis =0))**2, axis = 0) / float(X.shape[1] - 1)
f_m = msr / mse
pf_m = 1 - stats.f.cdf(np.abs(f_m), X.shape[1] -1, X.shape[0] - X.shape[1])

In [36]:
Sig_b = np.where(pf_m < 0.05/m_values.shape[0]) # Bonferonni?

keys = list(site_id[i] for i in Sig_b[0])
Features = ['age', 'bmi', 'smoking factor 0', 'smoking factor 1', 'smoking factor 2']
Sig_f = np.where(p_m[Sig_b] < 0.05)
factors = list(Features[i] for i in Sig_f[1])
cites = list(keys[i] for i in Sig_f[0])
coef = list(lr_m.coef_[Sig_b][Sig_f[0][i],Sig_f[1][i]] for i in range(len(Sig_f[0])))
p_value = list(p_m[Sig_b][Sig_f[0][i],Sig_f[1][i]] for i in range(len(Sig_f[0])))
result = np.stack((cites, factors, coef, p_value)).T

In [40]:
result[5]

array(['cg00868131', 'smoking factor 0', '-0.5455663777236318',
       '0.0027743535820747844'], 
      dtype='<U32')

In [42]:
with open('/Users/papiya/Desktop/CU_Capstone/Dataset/result_1.csv', 'w', newline='') as file:
    for line in result:
        file.write(str(','.join(list(line))+'\n'))